In [1]:
# Program places orders

# STATUS: Incomplete

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=1) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=1) # kavi IBG live

In [2]:
import pandas as pd
import datetime

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
base_rom = 1.25    #90% RoM atleast

df_orders = pd.read_pickle(datapath+'df_orders.pkl')

# For puts place orders for strikes below threshold
df_orders = df_orders[df_orders.Strike <= df_orders.Threshold]

# Filter out the top Return on Margin trades
df_orders1 = df_orders[df_orders.RoM >= base_rom].reset_index(drop=True)
df_orders1 = df_orders1.sort_values('RoM', ascending=False)

In [3]:
df_orders1 = df_orders1[df_orders1.RoM > 100]

In [4]:
df_orders1

,ibSymbol,undPrice,Strike,Expiry,DTE,pLTP,pBidPrice,tgt_price,Type,Mlot,Delta,RoM,Threshold,Contract,Orders
0,NHPC,26.45,16.0,20181227,20,5.00,NaN,5.00,P,27000,1.000000,1361.836794,21.0,"Option(conId=335444651, symbol='NHPC', lastTra...",LimitOrder(softDollarTier=1621240603144: Name:...
1,CGPOWER,37.55,20.0,20181227,20,2.95,NaN,2.95,P,12000,1.000000,557.051300,30.0,"Option(conId=335613548, symbol='CGPOWER', last...",LimitOrder(softDollarTier=1621240603144: Name:...
2,NHPC,26.45,13.0,20181227,20,1.70,NaN,1.70,P,27000,1.000000,463.024510,21.0,"Option(conId=335444626, symbol='NHPC', lastTra...",LimitOrder(softDollarTier=1621240603144: Name:...
3,NIFTY50,10601.15,7400.0,20181227,20,310.00,NaN,310.00,P,75,1.000000,367.715162,9950.0,"Option(conId=141093781, symbol='NIFTY50', last...",LimitOrder(softDollarTier=1621240603144: Name:...
4,SREINFRA,30.30,22.5,20181227,20,2.05,0.20,2.05,P,7000,0.999953,356.385973,25.0,"Option(conId=335396454, symbol='SREINFRA', las...",LimitOrder(softDollarTier=1621240603144: Name:...
5,BHEL,65.70,47.5,20181227,20,3.00,NaN,3.00,P,7500,1.000000,328.767123,62.5,"Option(conId=335418631, symbol='BHEL', lastTra...",LimitOrder(softDollarTier=1621240603144: Name:...
6,IDBI,60.20,35.0,20181227,20,2.65,NaN,2.65,P,10000,1.000000,315.680798,47.0,"Option(conId=335366236, symbol='IDBI', lastTra...",LimitOrder(softDollarTier=1621240603144: Name:...
7,UNIONBANK,73.55,47.5,20181227,20,2.70,NaN,2.70,P,6000,1.000000,263.556973,60.0,"Option(conId=335410582, symbol='UNIONBANK', la...",LimitOrder(softDollarTier=1621240603144: Name:...
8,NIFTY50,10601.15,6800.0,20181227,20,160.00,NaN,160.00,P,75,1.000000,189.788471,9950.0,"Option(conId=141093733, symbol='NIFTY50', last...",LimitOrder(softDollarTier=1621240603144: Name:...
9,CASTROLIN,147.50,125.0,20181227,20,3.45,NaN,3.45,P,2800,1.000000,168.022728,130.0,"Option(conId=335405453, symbol='CASTROLIN', la...",LimitOrder(softDollarTier=1621240603144: Name:...


In [5]:
df_orders1.to_excel(datapath+'temp.xlsx')

In [6]:
#------------------------------------------------
#####    WARNING: THIS PLACES THE TRADE  #######
#________________________________________________

limitTrade = [ib.placeOrder(contract, order) 
              for contract, order in zip(df_orders1.Contract, df_orders1.Orders)]

Error 201, reqId 1069: Order rejected - reason:YOUR ORDER IS NOT ACCEPTED. IN ORDER TO OBTAIN THE DESIRED POSITION YOUR EQUITY WITH LOAN VALUE [18551222.15 INR] MUST EXCEED THE INITIAL MARGIN [18550776.54 INR] PLUS VALUATION UNCERTAINTY [9968.87 INR]


#------------------------------------------------
#####         Code to cancel all trades    ############
#________________________________________________
cancel = ib.reqGlobalCancel()


In [7]:
# Put placed orders to a dataframe
df_orders_placed = pd.DataFrame(limitTrade)

In [8]:
# Write orders placed to file
x  =  datapath + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_orders_placed.xlsx"
writer = pd.ExcelWriter(x)
df_orders_placed.to_excel(writer, 'all-options', index=False, header=None)
writer.save()

In [9]:
ib.disconnect()